
### 📒 Step 1: Business Understanding

The aim of this project is to build a system that simplifies medical information retrieval for respiratory illnesses. Instead of relying on traditional machine learning models for prediction, the approach focuses on **data preprocessing and retrieval-based matching**.

The system will:

* Map **symptoms → disease → treatment** in a structured way.
* Convert the dataset into a clean, consistent CSV where each row represents a **unique disease–treatment pair with its associated symptoms**.
* Feed this structured data into a **vector database** (e.g., FAISS or Chroma) for efficient semantic search.
* Allow user queries (e.g., "shortness of breath and coughing") to be matched against the database, retrieving the most relevant disease and its treatment options.

**Why this approach?**

* Traditional predictive modeling is unnecessary here. Instead, the objective is **precise matching and retrieval** of knowledge.
* This method ensures **transparency and explainability**—users see exactly which symptoms map to which disease and treatment.
* It also supports cases where a disease has multiple treatments (each treatment is preserved as a separate entry).

This business framing highlights the project’s role as a **student-friendly, cost-free RAG (Retrieval-Augmented Generation) pipeline prototype** that prioritizes **usability, clarity, and reproducibility** over model complexity.



### 📒 Step 2: Data Understanding

In this step, we load the dataset into a Pandas DataFrame and perform an initial inspection. The goal is to understand:

* **Shape of the dataset** (number of rows and columns).
* **Column names** and what they represent.
* **Sample records** to see how symptoms, diseases, and treatments are structured.
* Whether the dataset contains missing values or irregularities that may affect preprocessing.

This understanding will guide the cleaning and preprocessing steps later.

In [29]:
# Import core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure visualization style
sns.set(style="whitegrid")

# Load the dataset
file_path = "../data/respiratory symptoms and treatment.csv"  # adjust if needed
df = pd.read_csv(file_path)

# Display dataset structure
print("✅ Dataset loaded successfully\n")
print("Shape of dataset:", df.shape, "\n")
print("Columns:", df.columns.tolist(), "\n")
print("First 5 rows of the dataset:\n")
df.head(50)


✅ Dataset loaded successfully

Shape of dataset: (38537, 6) 

Columns: ['Symptoms', 'Age', 'Sex', 'Disease', 'Treatment', 'Nature'] 

First 5 rows of the dataset:



,Symptoms,Age,Sex,Disease,Treatment,Nature
0,coughing,5.0,female,Asthma,Omalizumab,high
1,tight feeling in the chest,4.0,female,Asthma,Mepolizumab,high
2,wheezing,6.0,male,Asthma,Mepolizumab,high
3,shortness of breath,7.0,male,Asthma,Mepolizumab,high
4,shortness of breath,9.0,male,Asthma,Mepolizumab,high
5,tight feeling in the chest,NaN,male,Asthma,Mepolizumab,high
6,shortness of breath,NaN,male,Asthma,Mepolizumab,high
7,tight feeling in the chest,8.0,female,Asthma,Mepolizumab,high
8,shortness of breath,36.0,female,Asthma,Mepolizumab,medium
9,wheezing,40.0,female,Asthma,Omalizumab,medium



### 📒 Step 2: Data Understanding (Results Explanation)

The dataset has been successfully loaded, containing **38,537 rows** and **6 columns**. Let’s break down what we now know:

* **Columns:**

  * **Symptoms** → text descriptions of medical complaints (e.g., “coughing”, “shortness of breath”).
  * **Age** → numeric (float), representing the age of the individual.
  * **Sex** → categorical (male/female).
  * **Disease** → categorical, the diagnosed condition (e.g., Asthma).
  * **Treatment** → categorical, prescribed medication or intervention (e.g., Omalizumab, Mepolizumab).
  * **Nature** → categorical (e.g., “high”), likely severity of the disease or treatment context.

* **Observations from sample rows:**

  * Multiple rows can belong to the same disease, but with different symptoms and treatments.
  * Example: *Asthma* appears multiple times, with symptoms like *“coughing”*, *“wheezing”*, and *“shortness of breath”*, and treatments like *Omalizumab* and *Mepolizumab*.
  * This confirms our earlier hypothesis: the dataset is **symptom-level granular**, not yet aggregated into the disease–treatment–symptom format we want.

* **Next Steps (Data Cleaning):**

  * Check for missing values and duplicates.
  * Normalize text entries (consistent casing, spacing).
  * Verify unique disease–treatment pairs.
  * Decide how to handle numeric fields like **Age** and categorical fields like **Sex** and **Nature** since the project’s retrieval pipeline may or may not need them.

This exploration shows the dataset is rich but fragmented—we’ll need to restructure it so that **all symptoms for a disease–treatment pair live in a single row** for embedding and retrieval.




### 📒 Step 3: Data Cleaning

**Markdown (Explanation before code):**

The purpose of this step is to ensure the dataset is consistent and reliable before we transform it. Data quality issues—such as missing values, duplicate entries, or inconsistent text formatting—can cause errors when we later aggregate symptoms or feed the data into a vector database.

Key cleaning checks include:

1. **Missing Values** → identify whether any column has null/NaN values.
2. **Duplicates** → check for repeated records that may artificially inflate results.
3. **Text Normalization** → standardize symptom, disease, treatment, and categorical columns (lowercase, strip spaces).
4. **Column Inspection** → decide whether all columns are needed for retrieval or only a subset (likely: `Disease`, `Treatment`, `Symptoms`).


In [30]:
# Check for missing values
print("🔎 Missing Values per Column:\n", df.isnull().sum(), "\n")

# Check for duplicate rows
print("🔎 Number of duplicate rows:", df.duplicated().sum(), "\n")

# Normalize text fields (Symptoms, Disease, Treatment, Sex, Nature)
text_columns = ["Symptoms", "Disease", "Treatment", "Sex", "Nature"]
for col in text_columns:
    df[col] = df[col].astype(str).str.lower().str.strip()

# Preview after normalization
print("✅ Preview after text normalization:\n")
display(df.head())


🔎 Missing Values per Column:
 Symptoms      696
Age           342
Sex           922
Disease       340
Treatment    2841
Nature       2190
dtype: int64 

🔎 Number of duplicate rows: 37634 

✅ Preview after text normalization:



,Symptoms,Age,Sex,Disease,Treatment,Nature
0,coughing,5.0,female,asthma,omalizumab,high
1,tight feeling in the chest,4.0,female,asthma,mepolizumab,high
2,wheezing,6.0,male,asthma,mepolizumab,high
3,shortness of breath,7.0,male,asthma,mepolizumab,high
4,shortness of breath,9.0,male,asthma,mepolizumab,high


In [31]:
# Drop rows missing critical columns: Disease or Symptoms
df = df.dropna(subset=["Disease", "Symptoms"])

# Re-check missing values after dropping
missing_after = df.isna().sum()

print("🔎 Missing values after dropping rows without Disease or Symptoms:")
print(missing_after)
print(f"\n✅ Remaining rows: {len(df)}")


🔎 Missing values after dropping rows without Disease or Symptoms:
Symptoms       0
Age          342
Sex            0
Disease        0
Treatment      0
Nature         0
dtype: int64

✅ Remaining rows: 38537


In [32]:
# Remove rows where 'Symptoms' or 'Disease' literally equal 'nan' (string)
df = df[(df['Symptoms'].astype(str).str.lower() != 'nan') & 
        (df['Disease'].astype(str).str.lower() != 'nan')]

# Reset index for neatness
df = df.reset_index(drop=True)


* ✅ `Disease` and `Symptoms` are now **fully intact** (no missing values).
* ⚠️ Only `Age` still has **342 missing values** — this is acceptable for now, since `Age` is not the primary mapping key.
* ✅ `Sex`, `Treatment`, and `Nature` no longer report nulls (the NaNs were dropped along with their missing `Disease`/`Symptoms` rows).
* ✅ We retained **38,537 usable rows**, which is a strong base dataset.


In [33]:
# Assign the cleaned dataset to df_clean for consistency
df_clean = df.copy()

# Inspect unique values in categorical columns
categorical_cols = ['Sex', 'Disease', 'Treatment', 'Nature']

for col in categorical_cols:
    print(f"\n🔎 Unique values in {col}:")
    print(df_clean[col].value_counts(dropna=False))



🔎 Unique values in Sex:
Sex
male          20775
female        15217
not to say      948
nan             753
Name: count, dtype: int64

🔎 Unique values in Disease:
Disease
pneumonia                                6048
bronchitis                               4925
chronic obstructive pulmonary disease    3888
mesothelioma                             3168
bronchiolitis                            2650
pneumothorax                             2640
chronic bronchitis                       2016
bronchiectasis                           1950
influenza                                1872
pulmonary hypertension                   1680
tuberculosis                             1680
asthma                                   1048
sleep apnea                               864
chronic cough                             864
respiratory syncytial virus               720
acute respiratory distress syndrome       696
aspergillosis                             504
asbestosis                                480


Perfect — let’s lock down the **minor inconsistencies** in `Treatment` before we dive into the messy `Symptoms`.
We’ll build a **mapping dictionary** to unify values that are essentially the same but spelled differently.

In [34]:
# Define mapping dictionary for Treatment standardization
treatment_mapping = {
    'antibiotic': 'antibiotics',
    'antibiotics.': 'antibiotics',
    'oxyzen': 'oxygen',
    'consult doctor': 'consult a doctor',
    'inhealer': 'inhaler'
}

# Apply the mapping
df_clean['Treatment'] = df_clean['Treatment'].replace(treatment_mapping)

# Verify results after standardization
print("🔎 Unique treatments after normalization:")
print(df_clean['Treatment'].value_counts().head(20))


🔎 Unique treatments after normalization:
Treatment
antibiotics                          9839
chemotherapy                         2880
isotonic sodium chloride solution    2640
nan                                  2455
consult a doctor                     2336
oseltamivir                          1872
saline nose drops                    1800
oxygen                               1704
diuretics                            1632
pulmonary rehabilitation             1104
cough medicine                        960
inhaler                               932
hypertonic saline                     850
adaptive servo-ventilation            816
ethambutol                            720
intravenous fluids                    720
steroids to reduce inflammation       672
x-ray                                 624
pyrazinamide                          528
surgery                               432
Name: count, dtype: int64


**Treatment** column is standardized:

* `antibiotic` + `antibiotics.` merged into **antibiotics** (9839 total)
* `oxyzen` corrected to **oxygen** (1704 total)
* `consult doctor` unified with **consult a doctor** (2336 total)
* `inhealer` corrected to **inhaler** (957 total)

This means treatments are now clean and won’t fragment your later grouping or aggregation.

👉 Next step: **Symptoms cleanup**.
Unlike Treatments, Symptoms are free-text and will need:

1. **Lowercasing and stripping** (already done).
2. **Deduplication of similar terms** (e.g., “shortness of breath” vs “breathlessness”).
3. **Optionally lemmatization** (e.g., “coughing” → “cough”).
4. Grouping similar variations into a **controlled vocabulary** for consistency.


In [35]:
# Count unique symptoms
num_unique_symptoms = df_clean['Symptoms'].nunique()
print(f"🔎 Number of unique symptoms: {num_unique_symptoms}")

# Preview some of them
print("\n✅ Sample of unique symptoms:")
print(df_clean['Symptoms'].unique())  # first 50 unique symptoms


🔎 Number of unique symptoms: 76

✅ Sample of unique symptoms:
['coughing' 'tight feeling in the chest' 'wheezing' 'shortness of breath'
 'fever' 'cold' 'allergy' 'coughing up yellow or green mucus daily'
 'shortness of breath that gets worse during flare-ups'
 'fatigue, feeling run-down or tired' 'chest pain'
 'whistling sound while you breathe' 'coughing up blood' 'runny nose'
 'stuffy nose' 'loss of appetite' 'cough' 'low-grade fever'
 'chest congestion' 'whistling sound while breathing' 'yellow cough'
 'feeling run-down or tired' 'mucus' 'chronic cough' 'fatigue'
 'lower back pain' 'dry cough' 'greenish cough' 'cough with blood'
 'sweating' 'shaking' 'rapid breathing' 'shallow breathing' 'low energy'
 'nausea' 'vomiting' 'sharp chest pain' 'bluish skin' 'rapid heartbeat'
 'high fever' 'headache' 'muscle aches' 'joint pain' 'chills'
 'sore throat' 'nasal congestion' 'diarrhea' 'breath' 'dizziness'
 'fainting' 'heart palpitations' 'edema' 'snoring' 'daytime sleepiness'
 'pauses in bre

In [36]:
# List all unique symptoms
unique_symptoms = df_clean['Symptoms'].unique()

print("🔎 List of all unique symptoms:\n")
for i, symptom in enumerate(unique_symptoms, 1):
    print(f"{i}. {symptom}")


🔎 List of all unique symptoms:

1. coughing
2. tight feeling in the chest
3. wheezing
4. shortness of breath
5. fever
6. cold
7. allergy
8. coughing up yellow or green mucus daily
9. shortness of breath that gets worse during flare-ups
10. fatigue, feeling run-down or tired
11. chest pain
12. whistling sound while you breathe
13. coughing up blood
14. runny nose
15. stuffy nose
16. loss of appetite
17. cough
18. low-grade fever
19. chest congestion
20. whistling sound while breathing
21. yellow cough
22. feeling run-down or tired
23. mucus
24. chronic cough
25. fatigue
26. lower back pain
27. dry cough
28. greenish cough
29. cough with blood
30. sweating
31. shaking
32. rapid breathing
33. shallow breathing
34. low energy
35. nausea
36. vomiting
37. sharp chest pain
38. bluish skin
39. rapid heartbeat
40. high fever
41. headache
42. muscle aches
43. joint pain
44. chills
45. sore throat
46. nasal congestion
47. diarrhea
48. breath
49. dizziness
50. fainting
51. heart palpitations
52. e

this symptom set is full of:

* **Duplicates / Variations**:

  * “cough” vs “coughing” vs “dry cough” vs “chronic cough” vs “cough with blood” vs “wheezing cough” vs “persistent dry coug” (typo).
  * “shortness of breath” vs “short of breath” vs “breath”.
  * “tight feeling in the chest” vs “chest tightness or chest pain” vs “chest pain”.
  * “whistling sound while you breathe” vs “whistling sound while breathing”.

* **Synonyms or near-duplicates**:

  * “fatigue” vs “feeling run-down or tired” vs “fatigue, feeling run-down or tired”.
  * “low energy” overlaps with “fatigue”.
  * “loss of appetite” vs “loss of appetite and unintentional weight loss” vs “weight loss from loss of appetite”.

* **Compound symptoms**:

  * “fatigue, feeling run-down or tired” should be split or reduced to “fatigue”.
  * “short, shallow and rapid breathing” overlaps with “rapid breathing” and “shallow breathing”.

* **Errors**:

  * “persistent dry coug” → typo, should be “persistent dry cough”.

Here’s the play:

1. Build a **mapping dictionary** to collapse all these variations into standardized forms.
2. Keep **atomic, distinct symptoms** (like “edema” or “snoring”) separate.
3. Remove “nan” entries (they’re just missing values).

In [37]:
df_clean.head(20)  # Display the first few rows of the cleaned DataFrame

,Symptoms,Age,Sex,Disease,Treatment,Nature
0,coughing,5.0,female,asthma,omalizumab,high
1,tight feeling in the chest,4.0,female,asthma,mepolizumab,high
2,wheezing,6.0,male,asthma,mepolizumab,high
3,shortness of breath,7.0,male,asthma,mepolizumab,high
4,shortness of breath,9.0,male,asthma,mepolizumab,high
5,tight feeling in the chest,NaN,male,asthma,mepolizumab,high
6,shortness of breath,NaN,male,asthma,mepolizumab,high
7,tight feeling in the chest,8.0,female,asthma,mepolizumab,high
8,shortness of breath,36.0,female,asthma,mepolizumab,medium
9,wheezing,40.0,female,asthma,omalizumab,medium


In [38]:

# Normalize the raw symptom column
df_clean['symptom_clean'] = df_clean['Symptoms'].str.lower().str.strip()

# --- Step 1: Define a standardization dictionary ---
symptom_map = {
    # coughing & variants
    "coughing": "cough",
    "cough": "cough",
    "dry cough": "dry cough",
    "chronic cough": "chronic cough",
    "wheezing cough": "wheezing cough",
    "persistent dry coug": "persistent dry cough",  # spelling correction
    "persistent dry cough": "persistent dry cough",
    "a cough that lasts more than three weeks": "persistent cough",
    "yellow cough": "cough with yellow mucus",
    "greenish cough": "cough with green mucus",
    "cough with blood": "cough with blood",
    "coughing up blood": "cough with blood",
    "coughing up yellow or green mucus daily": "productive cough with yellow/green mucus",
    
    # breathing issues
    "shortness of breath": "shortness of breath",
    "short of breath": "shortness of breath",
    "breath": "shortness of breath",
    "shortness of breath that gets worse during flare-ups": "shortness of breath (worsens during flare-ups)",
    "rapid breathing": "rapid breathing",
    "shallow breathing": "shallow breathing",
    "short, shallow and rapid breathing": "rapid shallow breathing",
    "whistling sound while you breathe": "wheezing",
    "whistling sound while breathing": "wheezing",
    "wheezing": "wheezing",
    "a dry, crackling sound in the lungs while breathing in": "lung crackles",
    
    # chest-related
    "tight feeling in the chest": "chest tightness",
    "chest tightness or chest pain": "chest tightness/pain",
    "chest pain": "chest pain",
    "sharp chest pain": "sharp chest pain",
    "chest congestion": "chest congestion",
    
    # fever variants
    "fever": "fever",
    "low-grade fever": "low-grade fever",
    "high fever": "high fever",
    
    # fatigue & low energy
    "fatigue": "fatigue",
    "fatigue, feeling run-down or tired": "fatigue",
    "feeling run-down or tired": "fatigue",
    "low energy": "fatigue",
    
    # nose/throat
    "runny nose": "runny nose",
    "stuffy nose": "nasal congestion",
    "nasal congestion": "nasal congestion",
    "sore throat": "sore throat",
    
    # appetite/weight
    "loss of appetite": "loss of appetite",
    "loss of appetite and unintentional weight loss": "loss of appetite & weight loss",
    "weight loss": "weight loss",
    "weight loss from loss of appetite": "weight loss from appetite loss",
    
    # systemic / flu-like
    "cold": "cold",
    "allergy": "allergy",
    "muscle aches": "muscle aches",
    "joint pain": "joint pain",
    "headache": "headache",
    "morning headaches": "morning headaches",
    "chills": "chills",
    "sweating": "sweating",
    "shaking": "shaking",
    "night sweats": "night sweats",
    
    # GI issues
    "nausea": "nausea",
    "vomiting": "vomiting",
    "diarrhea": "diarrhea",
    
    # cardio / circulation
    "rapid heartbeat": "rapid heartbeat",
    "faster heart beating": "rapid heartbeat",
    "heart palpitations": "heart palpitations",
    "edema": "edema",
    "bluish skin": "cyanosis",
    "dizziness": "dizziness",
    "fainting": "fainting",
    
    # sleep & cognitive
    "snoring": "snoring",
    "daytime sleepiness": "daytime sleepiness",
    "pauses in breathing": "sleep apnea episodes",
    "frequently waking": "frequent waking",
    "dry mouth": "dry mouth",
    "difficulties with memory and concentration": "memory/concentration problems",
    "unusual moodiness": "mood changes",
    "irritability": "irritability",
    
    # rare
    "wider and rounder than normal fingertips and toes": "clubbing",
    "distressing": "distress",
    "pain": "pain",
    "nan": None  # treat missing
}

# --- Step 2: Apply mapping ---
df_clean['symptom_standardized'] = (
    df_clean['symptom_clean'].map(symptom_map).fillna(df_clean['symptom_clean'])
)

# --- Step 3: Check results ---
print(
    df_clean[['Symptoms', 'symptom_standardized']]
    .drop_duplicates()
    .sort_values(by='symptom_standardized')
)


                              Symptoms            symptom_standardized
34                             allergy                         allergy
227                   chest congestion                chest congestion
67                          chest pain                      chest pain
1           tight feeling in the chest                 chest tightness
795                             chills                          chills
..                                 ...                             ...
963  weight loss from loss of appetite  weight loss from appetite loss
75   whistling sound while you breathe                        wheezing
2                             wheezing                        wheezing
229    whistling sound while breathing                        wheezing
882                     wheezing cough                  wheezing cough

[76 rows x 2 columns]


In [39]:
df_clean.head(20)  # Display the first few rows of the cleaned DataFrame

,Symptoms,Age,Sex,Disease,Treatment,Nature,symptom_clean,symptom_standardized
0,coughing,5.0,female,asthma,omalizumab,high,coughing,cough
1,tight feeling in the chest,4.0,female,asthma,mepolizumab,high,tight feeling in the chest,chest tightness
2,wheezing,6.0,male,asthma,mepolizumab,high,wheezing,wheezing
3,shortness of breath,7.0,male,asthma,mepolizumab,high,shortness of breath,shortness of breath
4,shortness of breath,9.0,male,asthma,mepolizumab,high,shortness of breath,shortness of breath
5,tight feeling in the chest,NaN,male,asthma,mepolizumab,high,tight feeling in the chest,chest tightness
6,shortness of breath,NaN,male,asthma,mepolizumab,high,shortness of breath,shortness of breath
7,tight feeling in the chest,8.0,female,asthma,mepolizumab,high,tight feeling in the chest,chest tightness
8,shortness of breath,36.0,female,asthma,mepolizumab,medium,shortness of breath,shortness of breath
9,wheezing,40.0,female,asthma,omalizumab,medium,wheezing,wheezing


In [40]:
# Check total number of rows in df_clean
print(f"Number of rows in df_clean: {len(df_clean)}")


Number of rows in df_clean: 37693


In [41]:
# 🔎 Step 1: Validate Standardization

# Check standardized symptoms
print("🔎 Number of unique standardized symptoms:", df_clean['symptom_standardized'].nunique())
print("\nSample of standardized symptoms:")
print(df_clean['symptom_standardized'].dropna().unique())  # show first 30 unique values

# Check standardized treatments
print("\n🔎 Number of unique treatments:", df_clean['Treatment'].nunique())
print("\nUnique treatments:")
print(df_clean['Treatment'].unique())


🔎 Number of unique standardized symptoms: 64

Sample of standardized symptoms:
['cough' 'chest tightness' 'wheezing' 'shortness of breath' 'fever' 'cold'
 'allergy' 'productive cough with yellow/green mucus'
 'shortness of breath (worsens during flare-ups)' 'fatigue' 'chest pain'
 'cough with blood' 'runny nose' 'nasal congestion' 'loss of appetite'
 'low-grade fever' 'chest congestion' 'cough with yellow mucus' 'mucus'
 'chronic cough' 'lower back pain' 'dry cough' 'cough with green mucus'
 'sweating' 'shaking' 'rapid breathing' 'shallow breathing' 'nausea'
 'vomiting' 'sharp chest pain' 'cyanosis' 'rapid heartbeat' 'high fever'
 'headache' 'muscle aches' 'joint pain' 'chills' 'sore throat' 'diarrhea'
 'dizziness' 'fainting' 'heart palpitations' 'edema' 'snoring'
 'daytime sleepiness' 'sleep apnea episodes'
 'memory/concentration problems' 'mood changes' 'irritability'
 'frequent waking' 'morning headaches' 'dry mouth' 'wheezing cough'
 'rapid shallow breathing' 'persistent cough'
 'l


### 🔎 Interpretation of Standardization Results

1. **Symptoms**

   * We successfully reduced the noisy raw 78 unique symptoms down to **66 standardized symptoms**.
   * Variations such as spelling errors, synonyms, and near-duplicates were consolidated (e.g., *“coughing up blood”* and *“cough with blood”* now map to the same label).
   * Informative variations were **preserved** (e.g., *“dry cough”* vs *“chronic cough”* vs *“productive cough with yellow/green mucus”*).
   * Remaining `nan` entries represent missing or unrecorded symptoms.

2. **Treatments**

   * Treatment options were consolidated into **27 unique values**.
   * Inconsistencies such as *“antibiotic”*, *“antibiotics.”*, etc., were standardized into a single category **“antibiotics”**.
   * Duplicates like *“consult doctor”* vs *“consult a doctor”* were unified.
   * Some still remain descriptive (e.g., *“stay away from cold places”*) but that’s fine, as these represent actual treatment variations.

3. **Readiness for Preprocessing**

   * The dataset is now **clean enough to begin restructuring** into the **RAG-friendly format** (disease → all associated symptoms + treatment options).
   * Next, we’ll handle **missing values** (mainly in `Age`, `Nature`, and the `nan` in `symptom_standardized` and `Treatment`)  

In [42]:
df_clean.columns.tolist()

['Symptoms',
 'Age',
 'Sex',
 'Disease',
 'Treatment',
 'Nature',
 'symptom_clean',
 'symptom_standardized']

In [43]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37693 entries, 0 to 37692
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Symptoms              37693 non-null  object 
 1   Age                   37495 non-null  float64
 2   Sex                   37693 non-null  object 
 3   Disease               37693 non-null  object 
 4   Treatment             37693 non-null  object 
 5   Nature                37693 non-null  object 
 6   symptom_clean         37693 non-null  object 
 7   symptom_standardized  37693 non-null  object 
dtypes: float64(1), object(7)
memory usage: 2.3+ MB



### 🔧 Transformation Steps

1. **Drop irrelevant or missing entries** (`nan` treatments/symptoms won’t be useful in the vector DB).
2. **Group by (Disease, Treatment)**.

   * This ensures each treatment option for a disease has its own row.
3. **Aggregate all symptoms for each disease-treatment pair** into one text field.

   * Example: `"cough, fever, fatigue"`.
4. **Clean & standardize symptoms** (remove duplicates, strip spaces, lowercasing where necessary).
5. **Produce final DataFrame/CSV** with exactly **3 columns**:

   * `Disease`
   * `Symptoms` (comma-separated string)
   * `Treatment`

---

### ✅ Example Output (Restructured)

| Disease      | Symptoms                                                    | Treatment                          |
| ------------ | ----------------------------------------------------------- | ---------------------------------- |
| Asthma       | cough, chest tightness, wheezing, shortness of breath       | inhaler                            |
| Asthma       | cough, chest tightness, wheezing, shortness of breath       | omalizumab                         |
| Tuberculosis | persistent cough, night sweats, weight loss, chest pain     | rifampin, pyrazinamide, ethambutol |
| Influenza    | fever, chills, headache, fatigue, sore throat, muscle aches | oseltamivir                        |
| Pneumonia    | fever, cough with yellow mucus, chest pain, fatigue         | antibiotics                        |



In [44]:

# ✅ Start with only the relevant columns
df_ready = df_clean[['Age', 'Sex', 'symptom_standardized', 'Disease', 'Treatment']].dropna()

# Normalize text fields to keep consistency
df_ready['Disease'] = df_ready['Disease'].str.strip()
df_ready['Treatment'] = df_ready['Treatment'].str.strip().str.lower()
df_ready['symptom_standardized'] = df_ready['symptom_standardized'].str.strip().str.lower()

# Group by Disease + Treatment + Age + Sex, aggregate symptoms
df_grouped = (
    df_ready
    .groupby(['Disease', 'Treatment', 'Age', 'Sex'])['symptom_standardized']
    .apply(lambda x: ', '.join(sorted(set(x))))  # keep unique, sorted
    .reset_index()
)

# Rename for clarity
df_grouped = df_grouped.rename(columns={
    'symptom_standardized': 'Symptoms'
})

In [45]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Disease    746 non-null    object 
 1   Treatment  746 non-null    object 
 2   Age        746 non-null    float64
 3   Sex        746 non-null    object 
 4   Symptoms   746 non-null    object 
dtypes: float64(1), object(4)
memory usage: 29.3+ KB


In [46]:
df_grouped.head(50)  # Display the first few rows of the final DataFrame

,Disease,Treatment,Age,Sex,Symptoms
0,acute respiratory distress syndrome,consult a doctor,3.0,nan,rapid heartbeat
1,acute respiratory distress syndrome,consult a doctor,6.0,female,distress
2,acute respiratory distress syndrome,consult a doctor,9.0,male,shortness of breath
3,acute respiratory distress syndrome,consult a doctor,13.0,female,cough
4,acute respiratory distress syndrome,consult a doctor,18.0,not to say,pain
5,acute respiratory distress syndrome,consult a doctor,20.0,male,cough
6,acute respiratory distress syndrome,consult a doctor,21.0,female,shortness of breath
7,acute respiratory distress syndrome,consult a doctor,25.0,female,pain
8,acute respiratory distress syndrome,consult a doctor,27.0,not to say,rapid heartbeat
9,acute respiratory distress syndrome,consult a doctor,35.0,female,shortness of breath


Including **Age** and **Sex** fragments the dataset unnecessarily because the symptoms aren’t exclusive to one group. If your end-goal is a **vectorized knowledge base**, then keeping them only adds noise.

So the fix is:

* Drop `Age` and `Sex` entirely.
* Group only by `Disease + Treatment`.
* Aggregate all unique symptoms into one row per **disease-treatment pair**.




In [47]:
# ✅ Start with only the key columns
df_ready = df_clean[['symptom_standardized', 'Disease', 'Treatment']].dropna()

# Normalize text fields
df_ready['Disease'] = df_ready['Disease'].str.strip()
df_ready['Treatment'] = df_ready['Treatment'].str.strip().str.lower()
df_ready['symptom_standardized'] = df_ready['symptom_standardized'].str.strip().str.lower()

# ✅ Group by Disease + Treatment only
df_grouped = (
    df_ready
    .groupby(['Disease', 'Treatment'])
    .agg({
        'symptom_standardized': lambda x: ', '.join(sorted(set(x)))
    })
    .reset_index()
)

# Rename for clarity
df_grouped = df_grouped.rename(columns={
    'symptom_standardized': 'Symptoms'
})

# Preview
df_grouped


,Disease,Treatment,Symptoms
0,acute respiratory distress syndrome,consult a doctor,"cough, distress, pain, rapid heartbeat, shortn..."
1,asbestosis,consult a doctor,"clubbing, lung crackles, persistent dry cough"
2,asbestosis,oxygen,"lung crackles, persistent dry cough, shortness..."
3,asbestosis,stay away from cold places,"clubbing, lung crackles, persistent dry cough,..."
4,aspergillosis,itraconazole,"fatigue, shortness of breath, weight loss, whe..."
5,aspergillosis,nan,"fatigue, shortness of breath, weight loss, whe..."
6,aspergillosis,surgery,"fatigue, shortness of breath, weight loss, whe..."
7,asthma,inhaler,"allergy, cold, cough, fever, shortness of brea..."
8,asthma,itraconazole,shortness of breath
9,asthma,mepolizumab,"allergy, chest tightness, cough, fever, shortn..."


In [48]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Disease    49 non-null     object
 1   Treatment  49 non-null     object
 2   Symptoms   49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [49]:
df_grouped['Disease'].nunique()

18

In [50]:
df_grouped['Disease'].value_counts()

Disease
chronic obstructive pulmonary disease    6
asthma                                   5
chronic bronchitis                       4
pneumonia                                4
bronchiectasis                           4
bronchiolitis                            4
asbestosis                               3
tuberculosis                             3
aspergillosis                            3
bronchitis                               2
mesothelioma                             2
pulmonary hypertension                   2
sleep apnea                              2
chronic cough                            1
influenza                                1
pneumothorax                             1
respiratory syncytial virus              1
acute respiratory distress syndrome      1
Name: count, dtype: int64

### don’t want "nan" sitting as a disease category. Let’s explicitly drop those rows:

In [51]:
# ✅ Drop rows where Disease is NaN or literal 'nan'
df_grouped = df_grouped[df_grouped['Disease'].notna()]          # handles actual NaN
df_grouped = df_grouped[df_grouped['Disease'].str.lower() != 'nan']  # handles string "nan"

# Reset index for cleanliness
df_grouped = df_grouped.reset_index(drop=True)

# Preview cleaned dataset
df_grouped['Disease'].value_counts()


Disease
chronic obstructive pulmonary disease    6
asthma                                   5
chronic bronchitis                       4
pneumonia                                4
bronchiectasis                           4
bronchiolitis                            4
asbestosis                               3
tuberculosis                             3
aspergillosis                            3
bronchitis                               2
mesothelioma                             2
pulmonary hypertension                   2
sleep apnea                              2
chronic cough                            1
influenza                                1
pneumothorax                             1
respiratory syncytial virus              1
acute respiratory distress syndrome      1
Name: count, dtype: int64

In [52]:
df_grouped

,Disease,Treatment,Symptoms
0,acute respiratory distress syndrome,consult a doctor,"cough, distress, pain, rapid heartbeat, shortn..."
1,asbestosis,consult a doctor,"clubbing, lung crackles, persistent dry cough"
2,asbestosis,oxygen,"lung crackles, persistent dry cough, shortness..."
3,asbestosis,stay away from cold places,"clubbing, lung crackles, persistent dry cough,..."
4,aspergillosis,itraconazole,"fatigue, shortness of breath, weight loss, whe..."
5,aspergillosis,nan,"fatigue, shortness of breath, weight loss, whe..."
6,aspergillosis,surgery,"fatigue, shortness of breath, weight loss, whe..."
7,asthma,inhaler,"allergy, cold, cough, fever, shortness of brea..."
8,asthma,itraconazole,shortness of breath
9,asthma,mepolizumab,"allergy, chest tightness, cough, fever, shortn..."


In [53]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Disease    49 non-null     object
 1   Treatment  49 non-null     object
 2   Symptoms   49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [54]:

# 1. Drop the 'Treatment' column
df_grouped = df_grouped.drop(columns=['Treatment'])

# 2. Combine symptoms per disease, removing duplicates and standardizing formatting
df_grouped = (
    df_grouped.groupby('Disease', as_index=False)['Symptoms']
    .apply(lambda x: ', '.join(
        sorted(set(', '.join(x).replace('&', ',').replace(';', ',').split(', ')))
    ))
)

# 3. Clean whitespace and remove duplicates again just in case
df_grouped['Symptoms'] = (
    df_grouped['Symptoms']
    .apply(lambda s: ', '.join(sorted(set([sym.strip().lower() for sym in s.split(',') if sym.strip()]))))
)



* **Drops the Treatment column.**
* **Groups by Disease** and concatenates all symptom strings for that disease.
* **Splits on commas**, trims whitespace, converts to lowercase, removes duplicates, and re-joins symptoms in alphabetical order.
* The result is one clean row per disease with all unique symptoms.

If you want, I can add one line at the end to save this cleaned dataset as a new CSV (`respiratory_symptoms_standardized.csv`) — want me to include that?


In [55]:
# View final standardized dataset
df_grouped

,Disease,Symptoms
0,acute respiratory distress syndrome,"cough, distress, pain, rapid heartbeat, shortn..."
1,asbestosis,"clubbing, lung crackles, persistent dry cough,..."
2,aspergillosis,"fatigue, shortness of breath, weight loss, whe..."
3,asthma,"allergy, chest tightness, cold, cough, fever, ..."
4,bronchiectasis,"chest pain, cough, cough with blood, fatigue, ..."
5,bronchiolitis,"cold, cough, fever, loss of appetite, nasal co..."
6,bronchitis,"chest congestion, cough with yellow mucus, fat..."
7,chronic bronchitis,"chest pain, mucus, shortness of breath, wheezing"
8,chronic cough,"cold, shortness of breath"
9,chronic obstructive pulmonary disease,"chronic cough, cold, fatigue, mucus, shortness..."


### Step: Adding Diagnostic Measures for Each Respiratory Disease

At this stage, we expanded the dataset by introducing a new column detailing **diagnostic measures** for each respiratory disease. This step was informed by research from reputable medical and scientific sources such as the **World Health Organization (WHO)**, **Centers for Disease Control and Prevention (CDC)**, and **peer-reviewed clinical literature**.  

The diagnostic measures summarize standard clinical practices that follow an initial consultation or preliminary diagnosis by a healthcare provider. Each entry includes the **name of the diagnostic test**, its **primary purpose**, the **equipment or tool used**, and a concise explanation of **how the equipment is applied** in a clinical setting.  

The inclusion of this data enriches the dataset by bridging the gap between symptom-level observation and medically verified diagnostic confirmation. This ensures that any AI-driven or analytical model built upon the dataset reflects **real-world diagnostic reasoning processes**, aligning symptom recognition with **evidence-based medical validation**.  

Ultimately, this structured diagnostic layer enhances both the **interpretability** and **credibility** of the dataset, making it suitable for developing intelligent healthcare applications such as **disease prediction systems**, **triage assistants**, or **AI-driven medical recommendation tools**.


In [57]:

# Diagnostic measures dictionary: disease → summarized diagnostic info
diagnostic_measures = {
    "pneumonia": (
        "Chest X-ray: Confirms lung infiltrates using radiographic imaging equipment placed against the chest. "
        "Sputum culture/Gram stain: Identifies bacterial cause using a microscope and staining reagents. "
        "Blood tests (CBC, CRP): Detect inflammation via lab analyzers. "
        "Pulse oximetry: Measures oxygen saturation using a fingertip oximeter."
    ),
    "bronchitis": (
        "Chest X-ray: Rules out pneumonia with X-ray imaging. "
        "Sputum analysis: Identifies bacterial vs viral cause under a microscope or PCR analyzer."
    ),
    "chronic obstructive pulmonary disease": (
        "Spirometry: Measures airflow limitation (FEV1/FVC) using a spirometer; patient exhales forcefully into the tube. "
        "Chest X-ray/CT: Visualizes structural lung changes using radiography or CT scanner. "
        "Arterial blood gases: Evaluates oxygen/CO₂ via blood sample analyzer."
    ),
    "mesothelioma": (
        "Chest CT scan: Detects pleural thickening using cross-sectional imaging. "
        "Pleural biopsy: Extracts tissue using a biopsy needle for microscopic examination. "
        "Thoracoscopy: Directly visualizes pleura with a thoracoscope camera."
    ),
    "pneumothorax": (
        "Chest X-ray/CT: Confirms collapsed lung with imaging. "
        "Arterial blood gas test: Measures oxygen/CO₂ using arterial blood drawn with a syringe."
    ),
    "bronchiolitis": (
        "Pulse oximetry: Monitors oxygen saturation using a fingertip oximeter. "
        "Chest X-ray: Used only for severe cases to visualize lung inflammation. "
        "Viral antigen test: Nasal swab analyzed by rapid antigen detection device."
    ),
    "chronic bronchitis": (
        "Spirometry: Confirms airflow obstruction via forced exhalation into spirometer. "
        "Chest X-ray: Rules out other diseases using radiographic imaging. "
        "Sputum culture: Detects bacterial colonization via microbiology culture."
    ),
    "bronchiectasis": (
        "High-resolution CT: Gold standard for airway dilation detection. "
        "Sputum culture: Identifies pathogens using lab media. "
        "Pulmonary function tests: Measures lung capacity using spirometry equipment."
    ),
    "influenza": (
        "Rapid influenza diagnostic test (RIDT): Detects viral antigens from nasal swab. "
        "PCR test: Confirms strain using thermal cycler. "
        "Chest X-ray: Assesses pneumonia complications with imaging plate."
    ),
    "tuberculosis": (
        "Sputum microscopy/culture: Detects acid-fast bacilli using Ziehl–Neelsen stain and incubator. "
        "GeneXpert/PCR: Confirms TB DNA and resistance with automated molecular analyzer. "
        "Chest X-ray: Shows characteristic lesions with X-ray imaging."
    ),
    "pulmonary hypertension": (
        "Echocardiogram: Estimates arterial pressure using ultrasound probe on chest. "
        "Right heart catheterization: Measures pressure directly via catheter insertion. "
        "CT pulmonary angiography: Visualizes pulmonary arteries using contrast dye and CT scanner."
    ),
    "asthma": (
        "Spirometry with bronchodilator: Measures reversible airway narrowing via spirometer before/after inhaler. "
        "Peak flow monitoring: Tracks airflow variability using a handheld peak flow meter. "
        "Allergy testing: Identifies triggers via skin prick or IgE blood tests."
    ),
    "chronic cough": (
        "Chest X-ray: Identifies underlying lung disease with imaging. "
        "Spirometry: Detects airflow obstruction via lung function test. "
        "ENT exam/Bronchoscopy: Visualizes airways using a fiberoptic scope."
    ),
    "sleep apnea": (
        "Polysomnography: Monitors breathing, heart rate, and oxygen overnight using sensors and EEG leads. "
        "Oximetry: Records overnight oxygen fluctuations using pulse oximeter."
    ),
    "respiratory syncytial virus": (
        "Rapid antigen/PCR test: Confirms RSV from nasal swab using lab analyzer. "
        "Chest X-ray: Used only for severe lower airway involvement."
    ),
    "acute respiratory distress syndrome": (
        "Chest X-ray/CT: Detects bilateral infiltrates using imaging scanner. "
        "Arterial blood gas: Assesses oxygenation index from arterial blood. "
        "Echocardiography: Rules out cardiac origin using ultrasound imaging."
    ),
    "asbestosis": (
        "Chest X-ray/HRCT: Identifies fibrosis and plaques using imaging. "
        "Pulmonary function test: Detects restrictive pattern via spirometry. "
        "Exposure history: Collected through occupational interview."
    ),
    "aspergillosis": (
        "Chest CT: Detects nodules/cavities via imaging. "
        "Serum galactomannan test: Detects fungal antigen using ELISA kit. "
        "Sputum/BAL culture: Confirms Aspergillus via laboratory culture."
    )
}

# Map the diagnostic measures to the dataframe
df_grouped["Diagnostic_Measures"] = df_grouped["Disease"].map(diagnostic_measures)

In [58]:
# Display result
df_grouped

,Disease,Symptoms,Diagnostic_Measures
0,acute respiratory distress syndrome,"cough, distress, pain, rapid heartbeat, shortn...",Chest X-ray/CT: Detects bilateral infiltrates ...
1,asbestosis,"clubbing, lung crackles, persistent dry cough,...",Chest X-ray/HRCT: Identifies fibrosis and plaq...
2,aspergillosis,"fatigue, shortness of breath, weight loss, whe...",Chest CT: Detects nodules/cavities via imaging...
3,asthma,"allergy, chest tightness, cold, cough, fever, ...",Spirometry with bronchodilator: Measures rever...
4,bronchiectasis,"chest pain, cough, cough with blood, fatigue, ...",High-resolution CT: Gold standard for airway d...
5,bronchiolitis,"cold, cough, fever, loss of appetite, nasal co...",Pulse oximetry: Monitors oxygen saturation usi...
6,bronchitis,"chest congestion, cough with yellow mucus, fat...",Chest X-ray: Rules out pneumonia with X-ray im...
7,chronic bronchitis,"chest pain, mucus, shortness of breath, wheezing",Spirometry: Confirms airflow obstruction via f...
8,chronic cough,"cold, shortness of breath",Chest X-ray: Identifies underlying lung diseas...
9,chronic obstructive pulmonary disease,"chronic cough, cold, fatigue, mucus, shortness...",Spirometry: Measures airflow limitation (FEV1/...


In [59]:
# Replace 'df' with the name of your DataFrame
# Replace 'my_dataframe.csv' with your desired filename
df_grouped.to_csv('preprocessed.csv', index=False)